# Tarea 2 - Named Entity Recognition

----------------------

- **Nombre:** Ignacio Díaz Lara, Felipe Manen Núñez

- **Usuario o nombre de equipo en Codalab:** Equipo7



-----------------------------------------

## Introducción

En el presente informe se registra el trabajo realizado para el Sequence Labelling, en particular Named Entity Recognition (NER), sobre un dataset de noticias en español etiquetadas. La tarea por abordar consiste en asignar una etiqueta dada una secuencia de tokens proveniente de una oración. Dicha etiqueta busca reconocer entidades nombradas, es decir, clasificar los tokens que simbolicen personas (clasificadas como PER), organizaciones (ORG), lugares (LOC) y adjetivos, eventos y otras entidades que no entren en las categorías (MISC). Además, como existen entidades representadas en más de un token, se usa como prefijo del tag la notación BIO: Beginning, Inside, Outside. En esta notación, el primer token de una entidad tiene el prefijo B y todos los restantes de la misma entidad tienen el prefijo I. Si no representa ninguna entidad, entonces se usa el prefijo O. 

Para resolver esta tarea se utilizan Redes Neuronales Recurrentes (RNN). Para esto, el equipo de desarrollo construye diferentes modelos que componen una arquitectura de RNN en Pytorch, y a su vez, varios experimentos que combinan diferentes modelos, épocas, tamaños de batches, embeddings preentrenados, entre otras variables; de tal manera de poder obtener resultados aceptables a la hora de compararlos con los datos etiquetados según las métricas de evaluación dispuestas. Las métricas que se utilizan para evaluar los resultados son Precision, Recall y F1 Score. Todo esto se construye sobre un baseline entregado por el equipo docente, el cual entrega resultados que buscamos mejorar.

Como principal conclusión se tiene que las arquitecturas con embeddings preentrenados obtuvieron peores resultados que las arquitecturas sin estos.
Sin embargo, se mejoraron los resultados inciales del baseline entregado por el equipo docente en las tres métricas a evaluar y se utilizaron diferentes herramientas para poder optmizar los resultados.


## Modelos 


En este informe se presentan cuatro arquitecturas sobre las cuales se experimentan combinando los hiperparámetros que las definien. Las RNN de la arquitectura están programadas en Pytorch, cargan los dataset y crean batches de texto y padding.
La primera es una RNN de tipo LSTM que no usa embeddings preentrenados. 
La segunda es una RNN de tipo GRU que tampoco usa embeddings preentrenados.
La tercera es una RNN de tipo LSTM que usa sí usa embeddings preentrenados.
Y la cuarta es una RNN de tipo GRU que también usa embeddings preentrenados. 
Los Embeddings preentrenados que se usaron fueron Glove y FastText en español.
Dentro de los hiperparámetros que se varian en este trabajo se encuentran, el tamaño de batch, la cantidad de épocas, la dimensión de los embeddings, la dimensión de las capas ocultas, la cantidad de capas ocultas, el dropout y la bidireccionalidad.

## Métricas de evaluación

**Recall:**

Recall o sensibilidad es la fracción de instancias que han sido recuperadas del total. Se calcula como la proporción de instancia recuperadas realmente positivas sobre el total de instancias positivas y se describe de la siguiente manera:

$Recall = \frac{TruePos }{TruePos +FalseNeg}$

Mientras el valor de Recall se acerque más a 1, significa que la cantidad de instancias recuperadas es más cercana al total. Si se acerca a 0, significa que los documentos obtenidos no tienen relevancia.

**Precision:**

Precision es la fracción de instancias que fueron correctamente identificadas, es decir que fueron recuperadas y son relevantes. Se calcula comparando la proporción entre resultados realmente positivos y negativos en los considerados positivos. y describe así:

$Precision = \frac{TruePos }{TruePos +FalsePos}$

Mientras el valor de Precisión sea más cercano a 1, significa que más relevantes son las instancias recuperadas. Por el contrario, mientras más cercano a 0, menos relevantes son.

**F1-Score:**

Existe un *tradeoff* entre ambas medidas y por eso se puede ponderar ambas en un nuevo valor llamado F1-Score que es la media armónica entre Precision y Recall. Dando igual importancia a ambos la forma del F1-Score es la siguiente:

$F_{1}=\frac{2* Precision* Recall}{Precision+Recall}$

Mientras el valor de F1-Score sea más cercano a 1, significa que más perfectos son sus valores de Precision y Recall, ya que son ponderados con la misma relevancia. Mientras que más baje de 1, menos precisos y correctos son los resultados


## Experimentos


El código que les entregaremos servirá de baseline para luego implementar mejores modelos. 
En general, el código asociado a la carga de los datos, las funciones de entrenamiento, de evaluación y la predicción de los datos de la competencia no deberían cambiar. 
Solo deben preocuparse de cambiar la arquitectura del modelo, sus hiperparámetros y reportar, lo cual lo pueden hacer en las subsecciones *modelos*.



###  Carga de datos y Preprocesamiento

Para cargar los datos y preprocesarlos usaremos la librería [`torchtext`](https://github.com/pytorch/text).
En particular usaremos su módulo `data`, el cual según su documentación original provee: 

    - Ability to describe declaratively how to load a custom NLP dataset that's in a "normal" format
    - Ability to define a preprocessing pipeline
    - Batching, padding, and numericalizing (including building a vocabulary object)
    - Wrapper for dataset splits (train, validation, test)


El proceso será el siguiente: 

1. Descargar los datos desde github y examinarlos.
2. Definir los campos (`fields`) que cargaremos desde los archivos.
3. Cargar los datasets.
4. Crear el vocabulario.



In [2]:
# Instalar torchtext (en codalab) - Descomentar.
#!pip3 install --upgrade torchtext
!pip install torchtext==0.6

In [3]:
import torch
from torchtext import data, datasets


# Garantizar reproducibilidad 
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

#### Obtener datos

Descargamos los datos de entrenamiento, validación y prueba en nuestro directorio de trabajo

In [4]:
%%capture
!wget https://github.com/dccuchile/CC6205/releases/download/Data/train_NER_esp.txt -nc # Dataset de Entrenamiento
!wget https://github.com/dccuchile/CC6205/releases/download/Data/val_NER_esp.txt -nc    # Dataset de Validación (Para probar y ajustar el modelo)
!wget https://github.com/dccuchile/CC6205/releases/download/Data/test_NER_esp.txt -nc  # Dataset de la Competencia. Estos datos solo contienen los tokens. ¡¡SON LOS QUE DEBEN SER PREDICHOS!!

####  Fields

Un `field`:

* Define un tipo de datos junto con instrucciones para convertir el texto a Tensor.
* Contiene un objeto `Vocab` que contiene el vocabulario (palabras posibles que puede tomar ese campo).
* Contiene otros parámetros relacionados con la forma en que se debe numericalizar un tipo de datos, como un método de tokenización y el tipo de Tensor que se debe producir.


Analizemos el siguiente cuadro el cual contiene un ejemplo cualquiera de entrenamiento:


```
El O
Abogado B-PER
General I-PER
del I-PER
Estado I-PER
, O
Daryl B-PER
Williams I-PER
```

Cada linea contiene una palabra y su clase. Para que `torchtext` pueda cargar estos datos, debemos definir como va a leer y separar los componentes de cada una de las lineas.
Para esto, definiremos un field para cada uno de esos componentes: Las palabras (`TEXT`) y los NER_TAGS (`clase`).


In [5]:
# Primer Field: TEXT. Representan los tokens de la secuencia
TEXT = data.Field(lower=False) 

# Segundo Field: NER_TAGS. Representan los Tags asociados a cada palabra.
NER_TAGS = data.Field(unk_token=None)

fields = (("text", TEXT), ("nertags", NER_TAGS))

In [6]:
import csv
import gzip
import os
import shutil

import requests

GLOVE_FILE = "glove300d.vec"
# https://github.com/dccuchile/spanish-word-embeddings
if not os.path.exists(GLOVE_FILE):
    print(f"Descargando {GLOVE_FILE}")
    url = "http://dcc.uchile.cl/~jperez/word-embeddings/glove-sbwc.i25.vec.gz"
    response = requests.get(url, stream=True)
    try:
        with gzip.open(response.raw, "rb") as f_in:
            with open(GLOVE_FILE, "wb") as f_out:
                # Funcion util para copiar de un file-like object a otro
                shutil.copyfileobj(f_in, f_out)
    except Exception as e:
        os.remove(GLOVE_FILE)
        raise e

In [ ]:
import csv
import gzip
import os
import shutil

import requests

FASTTEXT_FILE = "fastText300d.vec"
# https://github.com/dccuchile/spanish-word-embeddings
if not os.path.exists(FASTTEXT_FILE):
    print(f"Descargando {FASTTEXT_FILE}")
    url = "http://dcc.uchile.cl/~jperez/word-embeddings/fasttext-sbwc.vec.gz"
    response = requests.get(url, stream=True)
    try:
        with gzip.open(response.raw, "rb") as f_in:
            with open(FASTTEXT_FILE, "wb") as f_out:
                # Funcion util para copiar de un file-like object a otro
                shutil.copyfileobj(f_in, f_out)
    except Exception as e:
        os.remove(FASTTEXT_FILE)
        raise e

Descargando fastText300d.vec


In [ ]:
from torchtext import vocab

# Para cargar los vectores de embeddings (que son esencialmente un vocabulario
# donde cada palabra tiene asociado un vector) pueden usar la clase vocab.Vectors
es_embeddings = vocab.Vectors(FASTTEXT_FILE)

####  SequenceTaggingDataset

`SequenceTaggingDataset` es una clase de torchtext diseñada para contener datasets de sequence labelling. 
Los ejemplos que se guarden en una instancia de estos serán arreglos de palabras pareados con sus respectivos tags.
Por ejemplo, para Part-of-speech tagging:

[I, love, PyTorch, .] estará pareado con [PRON, VERB, PROPN, PUNCT]


La idea es que usando los fields que definimos antes, le indiquemos a la clase cómo cargar los datasets de prueba, validación y test.

In [7]:
train_data, valid_data, test_data = datasets.SequenceTaggingDataset.splits(
    path="./",
    train="train_NER_esp.txt",
    validation="val_NER_esp.txt",
    test="test_NER_esp.txt",
    fields=fields,
    encoding="iso-8859-1",
    separator=" "
)

In [8]:
print(f"Numero de ejemplos de entrenamiento: {len(train_data)}")
print(f"Número de ejemplos de validación: {len(valid_data)}")
print(f"Número de ejemplos de test (competencia): {len(test_data)}")

Numero de ejemplos de entrenamiento: 8323
Número de ejemplos de validación: 1915
Número de ejemplos de test (competencia): 1517


Visualizemos un ejemplo

In [9]:
import random
random_item_idx = random.randint(0, len(train_data))
random_example = train_data.examples[random_item_idx]
list(zip(random_example.text, random_example.nertags))

[('El', 'O'),
 ('presidente', 'O'),
 ('de', 'O'),
 ('la', 'O'),
 ('FGF', 'B-ORG'),
 ('dijo', 'O'),
 ('que', 'O'),
 ('"', 'O'),
 ('Camacho', 'B-PER'),
 ('pretende', 'O'),
 ('recuperar', 'O'),
 ('a', 'O'),
 ('los', 'O'),
 ('jugadores', 'O'),
 ('y', 'O'),
 ('que', 'O'),
 ('descansen', 'O'),
 (',', 'O'),
 ('ese', 'O'),
 ('es', 'O'),
 ('el', 'O'),
 ('objetivo', 'O'),
 ('de', 'O'),
 ('la', 'O'),
 ('concentración', 'O'),
 ('de', 'O'),
 ('La', 'B-LOC'),
 ('Toja', 'I-LOC'),
 (',', 'O'),
 ('aunque', 'O'),
 ('hará', 'O'),
 ('algunos', 'O'),
 ('entrenamientos', 'O'),
 ('"', 'O'),
 ('.', 'O')]

#### Construir los vocabularios para el texto y las etiquetas

Los vocabularios son los obbjetos que contienen todos los tokens (de entrenamiento) posibles para ambos fields.
El siguiente paso consiste en construirlos. Para esto, hacemos uso del método `Field.build_vocab` sobre cada uno de nuestros `fields`. 

In [10]:
TEXT.build_vocab(train_data)
NER_TAGS.build_vocab(train_data)

In [ ]:
#para embeddings
from operator import attrgetter

TEXT.vocab.set_vectors(*attrgetter("stoi", "vectors", "dim")(es_embeddings))

In [11]:
print(f"Tokens únicos en TEXT: {len(TEXT.vocab)}")
print(f"Tokens únicos en NER_TAGS: {len(NER_TAGS.vocab)}")

Tokens únicos en TEXT: 26101
Tokens únicos en NER_TAGS: 10


In [12]:
#Veamos las posibles etiquetas que hemos cargado:
NER_TAGS.vocab.itos

['<pad>',
 'O',
 'B-ORG',
 'I-ORG',
 'B-LOC',
 'B-PER',
 'I-PER',
 'I-MISC',
 'B-MISC',
 'I-LOC']

Observen que ademas de los tags NER, tenemos \<pad\>, el cual es generado por el dataloader para cumplir con el padding de cada oración.

Veamos ahora los tokens mas frecuentes y especiales:

In [13]:
# Tokens mas frecuentes
TEXT.vocab.freqs.most_common(10)

[('de', 17657),
 (',', 14716),
 ('la', 9571),
 ('que', 7516),
 ('.', 7263),
 ('el', 6905),
 ('en', 6484),
 ('"', 5691),
 ('y', 5336),
 ('a', 4304)]

In [15]:
# Seteamos algunas variables que nos serán de utilidad mas adelante...
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

PAD_TAG_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]
O_TAG_IDX = NER_TAGS.vocab.stoi['O']

#### Frecuencia de los Tags

Visualizemos rápidamente las cantidades y frecuencias de cada tag:

In [16]:
def tag_percentage(tag_counts):
    
    total_count = sum([count for tag, count in tag_counts])
    tag_counts_percentages = [(tag, count, count/total_count) for tag, count in tag_counts]
  
    return tag_counts_percentages

print("Tag Ocurrencia Porcentaje\n")

for tag, count, percent in tag_percentage(NER_TAGS.vocab.freqs.most_common()):
    print(f"{tag}\t{count}\t{percent*100:4.1f}%")

Tag Ocurrencia Porcentaje

O	231920	87.6%
B-ORG	7390	 2.8%
I-ORG	4992	 1.9%
B-LOC	4913	 1.9%
B-PER	4321	 1.6%
I-PER	3903	 1.5%
I-MISC	3212	 1.2%
B-MISC	2173	 0.8%
I-LOC	1891	 0.7%


#### Configuramos pytorch y dividimos los datos.

Importante: si tienes problemas con la ram de la gpu, disminuye el tamaño de los batches

In [38]:
BATCH_SIZE = 64  # disminuir si hay problemas de ram.

# Usar cuda si es que está disponible.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using', device)

# Dividir datos entre entrenamiento y test
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device,
    sort=False,
)

Using cuda


#### Métricas de evaluación

Además, definiremos las métricas que serán usadas tanto para la competencia como para evaluar el modelo: `precision`, `recall` y `f1`.
**Importante**: Noten que la evaluación solo se hace para las Named Entities (sin contar 'O').

In [39]:
# Definimos las métricas

from sklearn.metrics import f1_score, precision_score, recall_score
import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore",
                        category=sklearn.exceptions.UndefinedMetricWarning)


def calculate_metrics(preds, y_true, pad_idx=PAD_TAG_IDX, o_idx=O_TAG_IDX):
    """
    Calcula precision, recall y f1 de cada batch.
    """

    # Obtener el indice de la clase con probabilidad mayor. (clases)
    y_pred = preds.argmax(dim=1, keepdim=True)
    # Obtenemos los indices distintos de 0.

    # filtramos <pad> y O para calcular los scores.
    mask = [(y_true != o_idx) & (y_true != pad_idx)]
    y_pred = y_pred[mask]
    y_true = y_true[mask]

    # traemos a la cpu
    y_pred = y_pred.view(-1).to('cpu')
    y_true = y_true.to('cpu')
    
    # calcular scores
    f1 = f1_score(y_true, y_pred, average='macro')
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')

    return precision, recall, f1

-------------------

### Modelo Baseline

Teniendo ya cargado los datos, toca definir nuestro modelo. Este baseline tendrá una capa de embedding, unas cuantas LSTM y una capa de salida y usará dropout en el entrenamiento.

Este constará de los siguientes pasos: 

1. Definir la clase que contendrá la red.
2. Definir los hiperparámetros e inicializar la red. 
3. Definir la época de entrenamiento
3. Definir la función de loss.



Recomendamos que para experimentar, encapsules los modelos en una sola variable y luego la fijes en model para entrenarla

In [40]:
#baseline
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


# Definir la red
class NER_RNN(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):

        super().__init__()

        # Capa de embedding
        self.embedding = nn.Embedding(input_dim,
                                      embedding_dim,
                                      padding_idx=pad_idx)

        # Capa LSTM
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        #text = [sent len, batch size]

        # Convertir lo enviado a embedding
        embedded = self.dropout(self.embedding(text))
        #outputs, hn = self.gru(embedded)
        outputs, (hidden, cell) = self.lstm(embedded)
        #embedded = [sent len, batch size, emb dim]

        # Pasar los embeddings por la rnn (LSTM)

        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]

        # Predecir usando la capa de salida.
        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]

        return predictions

In [20]:
#modelo con embedding preentrenados y gru
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


# Definir la red
class NER_RNN_pre_embeddings_LSTM(nn.Module):
    def __init__(self,
                 embedding_dim,  
                 embedding_weights,
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):

        super().__init__()

        # Capa de embedding
        #self.embedding = nn.Embedding(input_dim,
        #                              embedding_dim,
        #                              padding_idx=pad_idx)
        self.embedding = nn.Embedding.from_pretrained(
            embedding_weights.clone(), freeze=True
        )
        #self.fc = nn.Linear(embedding_weights.shape[1], num_class)
        # Capa LSTM
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        embedded = self.embedding(text)
        #return self.fc(embedded)
        #text = [sent len, batch size]

        # Convertir lo enviado a embedding
        #embedded = self.dropout(self.embedding(text))
        #outputs, hn = self.lstm(embedded)
        outputs, (hidden, cell) = self.lstm(embedded)
        #outputs, (hidden, cell) = self.lstm(embedded)
        #embedded = [sent len, batch size, emb dim]

        # Pasar los embeddings por la rnn (LSTM)

        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]

        # Predecir usando la capa de salida.
        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]

        return predictions

In [59]:
#RNN con GRU sin embedding preentrenados
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


# Definir la red
class NER_RNN_GRU(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):

        super().__init__()

        # Capa de embedding
        self.embedding = nn.Embedding(input_dim,
                                      embedding_dim,
                                      padding_idx=pad_idx)

        # Capa LSTM
        self.gru = nn.GRU(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        #text = [sent len, batch size]

        # Convertir lo enviado a embedding
        embedded = self.dropout(self.embedding(text))
        outputs, hn = self.gru(embedded)
        #outputs, (hidden, cell) = self.lstm(embedded)
        #embedded = [sent len, batch size, emb dim]

        # Pasar los embeddings por la rnn (LSTM)

        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]

        # Predecir usando la capa de salida.
        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]

        return predictions

In [ ]:
#modelo con embedding preentrenados y gru
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


# Definir la red
class NER_RNN_pre_embeddings(nn.Module):
    def __init__(self,
                 embedding_dim,  
                 embedding_weights,
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):

        super().__init__()

        # Capa de embedding
        #self.embedding = nn.Embedding(input_dim,
        #                              embedding_dim,
        #                              padding_idx=pad_idx)
        self.embedding = nn.Embedding.from_pretrained(
            embedding_weights.clone(), freeze=True
        )
        #self.fc = nn.Linear(embedding_weights.shape[1], num_class)
        # Capa LSTM
        self.gru = nn.GRU(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        embedded = self.embedding(text)
        #return self.fc(embedded)
        #text = [sent len, batch size]

        # Convertir lo enviado a embedding
        #embedded = self.dropout(self.embedding(text))
        outputs, hn = self.gru(embedded)
        #outputs, (hidden, cell) = self.lstm(embedded)
        #embedded = [sent len, batch size, emb dim]

        # Pasar los embeddings por la rnn (LSTM)

        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]

        # Predecir usando la capa de salida.
        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]

        return predictions

#### Hiperparámetros de la red

Definimos los hiperparámetros. 

In [41]:
#Hiperparámetros Baseline
# tamaño del vocabulario. recuerden que la entrada son vectores bag of word(one-hot).
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100  # dimensión de los embeddings.
HIDDEN_DIM = 256  # dimensión de la capas LSTM
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 5  # número de capas.
DROPOUT = 0.25
BIDIRECTIONAL = True

# Creamos nuestro modelo.
baseline_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

baseline_model_name = 'baseline'  # nombre que tendrá el modelo guardado..

In [ ]:
# Hiperparámetros NER_RNN_pre_embeddings
INPUT_DIM_1 = len(TEXT.vocab)
EMBEDDING_DIM_1 = 300  # dimensión de los embeddings.
HIDDEN_DIM_1 = 128  # dimensión de la capas LSTM
OUTPUT_DIM_1 = len(NER_TAGS.vocab)  # número de clases
EMBEDDING_WEIGHTS_1 = TEXT.vocab.vectors

N_LAYERS_1 = 5  # número de capas.
DROPOUT_1 = 0.1
BIDIRECTIONAL_1 = True

# Creamos nuestro modelo.
model_embeddings_1 = NER_RNN_pre_embeddings(EMBEDDING_DIM_1, EMBEDDING_WEIGHTS_1, HIDDEN_DIM_1, OUTPUT_DIM_1,
                         N_LAYERS_1, BIDIRECTIONAL_1, DROPOUT_1, PAD_IDX)

model_embeddings_name_1 = 'modelo NER_RNN_pre_embeddings'  # nombre que tendrá el modelo guardado...

In [228]:
# Hiperparámetros NER_RNN_GRU
INPUT_DIM_2 = len(TEXT.vocab)
EMBEDDING_DIM_2 = 300  # dimensión de los embeddings.
HIDDEN_DIM_2 = 128  # dimensión de la capas LSTM
OUTPUT_DIM_2 = len(NER_TAGS.vocab)  # número de clases


N_LAYERS_2 = 3  # número de capas.
DROPOUT_2 = 0.25
BIDIRECTIONAL_2 = True

# Creamos nuestro modelo.
model_embeddings_2 = NER_RNN_GRU(INPUT_DIM_2, EMBEDDING_DIM_2, HIDDEN_DIM_2, OUTPUT_DIM_2,
                         N_LAYERS_2, BIDIRECTIONAL_2, DROPOUT_2, PAD_IDX)

model_embeddings_name_2 = 'modelo NER_RNN_GRU sin embedding'  # nombre que tendrá el modelo guardado...

In [ ]:
# Hiperparámetros NER_RNN_pre_embeddings_LSTM
INPUT_DIM_3 = len(TEXT.vocab)
EMBEDDING_DIM_3 = 300  # dimensión de los embeddings.
HIDDEN_DIM_3 = 128  # dimensión de la capas LSTM
OUTPUT_DIM_3 = len(NER_TAGS.vocab)  # número de clases
EMBEDDING_WEIGHTS_3 = TEXT.vocab.vectors

N_LAYERS_3 = 5  # número de capas.
DROPOUT_3 = 0.1
BIDIRECTIONAL_3 = False

# Creamos nuestro modelo.
model_embeddings_3 = NER_RNN_pre_embeddings_LSTM(EMBEDDING_DIM_3, EMBEDDING_WEIGHTS_3, HIDDEN_DIM_3, OUTPUT_DIM_3,
                         N_LAYERS_3, BIDIRECTIONAL_3, DROPOUT_3, PAD_IDX)

model_embeddings_name_3 = 'modelo NER_RNN_pre_embeddings_LSTM'  # nombre que tendrá el modelo guardado...

In [131]:
baseline_n_epochs = 10

#### Definimos la función de loss

In [229]:
# Loss: Cross Entropy
TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]
baseline_criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)


--------------------
### Modelo 1

En estas secciones pueden implementar nuevas redes al modificar los hiperparámetros, la cantidad de épocas de entrenamiento, el tamaño de los batches, loss, optimizador, etc... como también definir nuevas arquitecturas de red (mediante la creación de clases nuevas)


Al final de estas, hay 4 variables, las cuales deben setear con los modelos, épocas de entrenamiento, loss y optimizador que deseen probar.


In [ ]:
model_1 = model_embeddings_1
model_name_1 = model_embeddings_name_1
n_epochs_1 = 20
loss_1 = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

---------------

### Modelo 2

In [135]:
model_2 = model_embeddings_2
model_name_2 = model_embeddings_name_2
n_epochs_2 = 10
loss_2 = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

---------------


### Modelo 3

In [ ]:
model_3 = model_embeddings_3
model_name_3 = model_embeddings_name_3
n_epochs_3 = 20
loss_3 = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

Modelo 4

In [230]:
model_4 = baseline_model
model_name_4 = baseline_model_name
n_epochs_4 = 15
loss_4 = baseline_criterion

------
### Entrenamos y evaluamos


**Importante** : Fijen el modelo, el número de épocas de entrenamiento, la loss y el optimizador que usarán para entrenar y evaluar en las siguientes variables!!!

In [231]:
model = model_2
model_name = model_name_2
criterion = loss_2
n_epochs = n_epochs_2



#### Inicializamos la red

iniciamos los pesos de la red de forma aleatoria (Usando una distribución normal).


In [232]:
def init_weights(m):
    # Inicializamos los pesos como aleatorios
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.1) 
        
    # Seteamos como 0 los embeddings de UNK y PAD.
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM_2)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM_2)
        
model.apply(init_weights)

NER_RNN_GRU(
  (embedding): Embedding(26101, 300, padding_idx=1)
  (gru): GRU(300, 256, num_layers=2, dropout=0.25, bidirectional=True)
  (fc): Linear(in_features=512, out_features=10, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

In [233]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')

El modelo actual tiene 9,875,238 parámetros entrenables.


Por último, definimos los embeddings que representan a \<unk\> y \<pad\>  como [0, 0, ..., 0]

#### Definimos el optimizador

In [234]:
# Optimizador
optimizer = optim.Adam(model.parameters())

#### Enviamos el modelo a cuda


In [235]:
# Enviamos el modelo y la loss a cuda (en el caso en que esté disponible)
model = model.to(device)
criterion = criterion.to(device)

#### Definimos el entrenamiento de la red

Algunos conceptos previos: 

- `epoch` : una pasada de entrenamiento completa de una dataset.
- `batch`: una fracción de la época. Se utilizan para entrenar mas rápidamente la red. (mas eficiente pasar n datos que uno en cada ejecución del backpropagation)

Esta función está encargada de entrenar la red en una época. Para esto, por cada batch de la época actual, predice los tags del texto, calcula su loss y luego hace backpropagation para actualizar los pesos de la red."

Observación: En algunos comentarios aparecerá el tamaño de los tensores entre corchetes

In [236]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.train()

    # Por cada batch del iterador de la época:
    for batch in iterator:

        # Extraemos el texto y los tags del batch que estamos procesado
        text = batch.text
        tags = batch.nertags

        # Reiniciamos los gradientes calculados en la iteración anterior
        optimizer.zero_grad()

        #text = [sent len, batch size]

        # Predecimos los tags del texto del batch.
        predictions = model(text)

        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]

        # Reordenamos los datos para calcular la loss
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)

        #predictions = [sent len * batch size, output dim]
        #tags = [sent len * batch size]

        # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
        loss = criterion(predictions, tags)
        
        # Calculamos el accuracy
        precision, recall, f1 = calculate_metrics(predictions, tags)

        # Calculamos los gradientes
        loss.backward()

        # Actualizamos los parámetros de la red
        optimizer.step()

        # Actualizamos el loss y las métricas
        epoch_loss += loss.item()
        epoch_precision += precision
        epoch_recall += recall
        epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(
        iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

#### `Definimos la función de evaluación`

Evalua el rendimiento actual de la red usando los datos de validación. 

Por cada batch de estos datos, calcula y reporta el loss y las métricas asociadas al conjunto de validación. 
Ya que las métricas son calculadas por cada batch, estas son retornadas promediadas por el número de batches entregados. (ver linea del return)

In [237]:
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.eval()

    # Indicamos que ahora no guardaremos los gradientes
    with torch.no_grad():
        # Por cada batch
        for batch in iterator:

            text = batch.text
            tags = batch.nertags

            # Predecimos
            predictions = model(text)

            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)

            # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
            loss = criterion(predictions, tags)

            # Calculamos las métricas
            precision, recall, f1 = calculate_metrics(predictions, tags)

            # Actualizamos el loss y las métricas
            epoch_loss += loss.item()
            epoch_precision += precision
            epoch_recall += recall
            epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(
        iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

In [238]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


#### Entrenamiento de la red

En este cuadro de código ejecutaremos el entrenamiento de la red.
Para esto, primero definiremos el número de épocas y luego por cada época, ejecutaremos `train` y `evaluate`.

**Importante: Reiniciar los pesos del modelo**

Si ejecutas nuevamente esta celda, se seguira entrenando el mismo modelo una y otra vez. 
Para reiniciar el modelo se debe ejecutar nuevamente la celda que contiene la función `init_weights`



In [239]:
best_valid_loss = float('inf')

for epoch in range(n_epochs):

    start_time = time.time()

    # Recuerdo: train_iterator y valid_iterator contienen el dataset dividido en batches.

    # Entrenar
    train_loss, train_precision, train_recall, train_f1 = train(
        model, train_iterator, optimizer, criterion)

    # Evaluar (valid = validación)
    valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
        model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # Si obtuvimos mejores resultados, guardamos este modelo en el almacenamiento (para poder cargarlo luego)
    # Si detienen el entrenamiento prematuramente, pueden cargar el modelo en el siguiente recuadro de código.
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '{}.pt'.format(model_name))
    # Si ya no mejoramos el loss de validación, terminamos de entrenar.

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(
        f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
    )
    print(
        f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
    )

Epoch: 01 | Epoch Time: 0m 6s
	Train Loss: 0.435 | Train f1: 0.24 | Train precision: 0.41 | Train recall: 0.19
	 Val. Loss: 0.274 |  Val. f1: 0.45 |  Val. precision: 0.61 | Val. recall: 0.41
Epoch: 02 | Epoch Time: 0m 6s
	Train Loss: 0.142 | Train f1: 0.62 | Train precision: 0.70 | Train recall: 0.58
	 Val. Loss: 0.222 |  Val. f1: 0.60 |  Val. precision: 0.72 | Val. recall: 0.56
Epoch: 03 | Epoch Time: 0m 6s
	Train Loss: 0.066 | Train f1: 0.76 | Train precision: 0.79 | Train recall: 0.74
	 Val. Loss: 0.195 |  Val. f1: 0.64 |  Val. precision: 0.72 | Val. recall: 0.60
Epoch: 04 | Epoch Time: 0m 6s
	Train Loss: 0.037 | Train f1: 0.82 | Train precision: 0.84 | Train recall: 0.81
	 Val. Loss: 0.236 |  Val. f1: 0.64 |  Val. precision: 0.72 | Val. recall: 0.60
Epoch: 05 | Epoch Time: 0m 6s
	Train Loss: 0.024 | Train f1: 0.85 | Train precision: 0.86 | Train recall: 0.84
	 Val. Loss: 0.249 |  Val. f1: 0.64 |  Val. precision: 0.72 | Val. recall: 0.62
Epoch: 06 | Epoch Time: 0m 6s
	Train Loss: 0.

**Importante**: Recuerden que el último modelo entrenado no es el mejor (probablemente esté *overfitteado*), si no el que guardamos con la menor loss del conjunto de validación.
Para cargar el mejor modelo entrenado, ejecuten la siguiente celda.

Este problema lo pueden solucionar con *early stopping*.

In [240]:
# cargar el mejor modelo entrenado.
model.load_state_dict(torch.load('{}.pt'.format(model_name)))

<All keys matched successfully>

In [241]:
# Limpiar ram de cuda
torch.cuda.empty_cache()

#### Evaluamos el set de validación con el modelo final

Estos son los resultados de predecir el dataset de evaluación con el *mejor* modelo entrenado.

In [242]:
valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
    model, valid_iterator, criterion)

print(
    f'Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
)

Val. Loss: 0.195 |  Val. f1: 0.64 | Val. precision: 0.72 | Val. recall: 0.60



### Predecir datos para la competencia

Ahora, a partir de los datos de **test** y nuestro modelo entrenado, predeciremos las etiquetas que serán evaluadas en la competencia.

In [194]:
def predict_labels(model, iterator, criterion, fields=fields):

    # Extraemos los vocabularios.
    text_field = fields[0][1]
    nertags_field = fields[1][1]
    tags_vocab = nertags_field.vocab.itos
    words_vocab = text_field.vocab.itos

    model.eval()

    predictions = []

    with torch.no_grad():

        for batch in iterator:

            text_batch = batch.text
            text_batch = torch.transpose(text_batch, 0, 1).tolist()

            # Predecir los tags de las sentences del batch
            predictions_batch = model(batch.text)
            predictions_batch = torch.transpose(predictions_batch, 0, 1)

            # por cada oración predicha:
            for sentence, sentence_prediction in zip(text_batch,
                                                     predictions_batch):
                for word_idx, word_predictions in zip(sentence,
                                                      sentence_prediction):
                    # Obtener el indice del tag con la probabilidad mas alta.
                    argmax_index = word_predictions.topk(1)[1]

                    current_tag = tags_vocab[argmax_index]
                    # Obtenemos la palabra
                    current_word = words_vocab[word_idx]

                    if current_word != '<pad>':
                        predictions.append([current_word, current_tag])


    return predictions


predictions = predict_labels(model, test_iterator, criterion)

### Generar el archivo para la submission

No hay problema si aparecen unk en la salida. Estos no son relevantes para evaluarlos, usamos solo los tags.

In [195]:
import os, shutil

if (os.path.isfile('./predictions.zip')):
    os.remove('./predictions.zip')

if (not os.path.isdir('./predictions')):
    os.mkdir('./predictions')

else:
    # Eliminar predicciones anteriores:

    shutil.rmtree('./predictions')
    os.mkdir('./predictions')

f = open('predictions/predictions.txt', 'w')
for word, tag in predictions:
    f.write(word + ' ' + tag + '\n')
f.write('\n')
f.close()

a = shutil.make_archive('predictions', 'zip', './predictions')

In [196]:
# A veces no funciona a la primera. Ejecutar mas de una vez para obtener el archivo...
from google.colab import files
files.download('predictions.zip')  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Experimentos y Resultados

Modelo RNN LSTM sin embedding preentrenados


|       | batches | emb_dim | hidden_dim | n_layers | dropout | bidirectional | epochs | Val Recall | Val Precision | Val F1 | Val Loss |
|-------|---------|---------|------------|----------|---------|---------------|--------|------------|---------------|--------|----------|
| Exp1  | 128     | 300     | 128        | 5        | 0,1     | VERDADERO     | 20     | 0,45       | 0,69          | 0,53   | 0,295    |
| Exp2  | 64      | 100     | 128        | 3        | 0,25    | FALSO         | 20     | 0,56       | 0,62          | 0,57   | 0,287    |
| Exp3  | 64      | 100     | 128        | 3        | 0,25    | VERDADERO     | 10     | 0,62       | 0,68          | 0,63   | 0,209    |
| Exp4  | 64      | 100     | 256        | 3        | 0,25    | VERDADERO     | 10     | 0,61       | 0,66          | 0,61   | 0,206    |
| Exp5  | 64      | 100     | 256        | 5        | 0,25    | VERDADERO     | 10     | 0,62       | 0,69          | 0,64   | 0,265    |
| Exp6  | 128     | 100     | 128        | 6        | 0,25    | VERDADERO     | 20     | 0,58       | 0,64          | 0,6    | 0,252    |
| Exp7  | 128     | 100     | 256        | 5        | 0,1     | VERDADERO     | 20     | 0,53       | 0,61          | 0,56   | 0,251    |
| Exp8  | 128     | 300     | 256        | 5        | 0,25    | VERDADERO     | 20     | 0,56       | 0,66          | 0,6    | 0,244    |
| Exp9  | 128     | 300     | 256        | 5        | 0,25    | VERDADERO     | 10     | 0,58       | 0,65          | 0,6    | 0,242    |
| Exp10 | 64      | 100     | 256        | 5        | 0,25    | VERDADERO     | 10     | 0,59       | 0,65          | 0,6    | 0,198    |


Mejores Resultados:

|      | batches | emb_dim | hidden_dim | n_layers | dropout | bidirectional | epochs | Val Recall | Val Precision | Val F1 | Val Loss |
|------|---------|---------|------------|----------|---------|---------------|--------|------------|---------------|--------|----------|
| Exp5 | 64      | 100     | 256        | 5        | 0,25    | VERDADERO     | 10     | 0,62       | 0,69          | 0,64   | 0,265    |

Modelo RNN GRU sin embedding preentrenado


|       | batches | emb_dim | hidden_dim | n_layers | dropout | bidirectional | epochs | Val Recall | Val Precision | Val F1 | Val Loss |
|-------|---------|---------|------------|----------|---------|---------------|--------|------------|---------------|--------|----------|
| Exp1  | 64      | 300     | 128        | 2        | 0,25    | VERDADERO     | 10     | 0,61       | 0,72          | 0,64   | 0,19     |
| Exp2  | 64      | 300     | 128        | 3        | 0,25    | VERDADERO     | 10     | 0,55       | 0,71          | 0,6    | 0,208    |
| Exp3  | 64      | 300     | 128        | 4        | 0,25    | VERDADERO     | 10     | 0,59       | 0,69          | 0,62   | 0,198    |
| Exp4  | 64      | 300     | 256        | 4        | 0,25    | VERDADERO     | 10     | 0,55       | 0,7           | 0,58   | 0,209    |
| Exp5  | 64      | 300     | 256        | 2        | 0,25    | VERDADERO     | 10     | 0,62       | 0,73          | 0,65   | 0,191    |
| Exp6  | 64      | 300     | 512        | 2        | 0,25    | VERDADERO     | 10     | 0,58       | 0,72          | 0,62   | 0,197    |
| Exp7  | 64      | 300     | 256        | 3        | 0,25    | VERDADERO     | 10     | 0,63       | 0,74          | 0,66   | 0,182    |
| Exp8  | 64      | 300     | 256        | 3        | 0,1     | VERDADERO     | 10     | 0,62       | 0,73          | 0,66   | 0,189    |
| Exp9  | 64      | 300     | 256        | 4        | 0,25    | VERDADERO     | 10     | 0,63       | 0,7           | 0,64   | 0,195    |
| Exp10 | 64      | 300     | 128        | 3        | 0,25    | VERDADERO     | 10     | 0,6        | 0,72          | 0,64   | 0,195    |

Mejores Resultados

|      | batches | emb_dim | hidden_dim | n_layers | dropout | bidirectional | epochs | Val Recall | Val Precision | Val F1 | Val Loss |
|------|---------|---------|------------|----------|---------|---------------|--------|------------|---------------|--------|----------|
| Exp7 | 64      | 300     | 256        | 3        | 0,25    | VERDADERO     | 10     | 0,63       | 0,74          | 0,66   | 0,182    |

Modelo RNN LSTM con embeddings preentrenados

|      | batches | emb_dim | hidden_dim |  | n_layers | dropout | bidirectional | epochs | freeze    | Embedding | Val Recall | Val Precission | Val F1 | Val Loss |
|------|---------|---------|------------|--|----------|---------|---------------|--------|-----------|-----------|------------|----------------|--------|----------|
| Exp1 | 30      | 300     | 128        |  | 4        | 0,25    | VERDADERO     | 10     | VERDADERO | Glove     | 0,56       | 0,68           | 0,59   | 0,241    |
| Exp2 | 100     | 300     | 256        |  | 5        | 0,5     | VERDADERO     | 10     | VERDADERO | Glove     | 0,45       | 0,66           | 0,52   | 0,297    |
| Exp3 | 100     | 300     | 128        |  | 5        | 0,15    | VERDADERO     | 10     | VERDADERO | Glove     | 0,54       | 0,66           | 0,58   | 0,268    |
| Exp4 | 100     | 300     | 64         |  | 4        | 0,25    | VERDADERO     | 20     | VERDADERO | Glove     | 0,53       | 0,66           | 0,57   | 0,253    |
| Exp5 | 30      | 300     | 128        |  | 7        | 0,25    | VERDADERO     | 20     | VERDADERO | Glove     | 0,56       | 0,69           | 0,59   | 0,245    |
| Exp6 | 30      | 300     | 128        |  | 4        | 0,25    | VERDADERO     | 20     | VERDADERO | FastText  | 0,54       | 0,66           | 0,57   | 0,247    |
| Exp7 | 30      | 300     | 256        |  | 5        | 0,5     | VERDADERO     | 10     | VERDADERO | FastText  | 0,53       | 0,66           | 0,56   | 0,262    |
| Exp8 | 30      | 300     | 128        |  | 5        | 0,15    | VERDADERO     | 10     | VERDADERO | FastText  | 0,55       | 0,67           | 0,57   | 0,241    |

Mejores Resultados

|      | batches | emb_dim | hidden_dim |  | n_layers | dropout | bidirectional | epochs | freeze    | Embedding | Val Recall | Val Precission | Val F1 | Val Loss |
|------|---------|---------|------------|--|----------|---------|---------------|--------|-----------|-----------|------------|----------------|--------|----------|
| Exp5 | 30      | 300     | 128        |  | 7        | 0,25    | VERDADERO     | 20     | VERDADERO | Glove     | 0,56       | 0,69           | 0,59   | 0,245    |

Modelo RNN GRU con embedding preentrenados

|      | batches | emb_dim | hidden_dim |   | n_layers | dropout | bidirectional | epochs | freeze    | Embedding | Val Recall | Val Precission | Val F1 | Val Loss |
|------|---------|---------|------------|---|----------|---------|---------------|--------|-----------|-----------|------------|----------------|--------|----------|
| Exp1 | 30      | 300     | 128        |   | 4        | 0,25    | VERDADERO     | 20     | VERDADERO | Glove     | 0,5        | 0,68           | 0,54   | 0,274    |
| Exp2 | 64      | 300     | 256        | 3 | 3        | 0,25    | VERDADERO     | 10     | VERDADERO | Glove     | 0,47       | 0,67           | 0,53   | 0,306    |
| Exp3 | 64      | 300     | 128        |   | 2        | 0,25    | VERDADERO     | 10     | VERDADERO | Glove     | 0,48       | 0,68           | 0,52   | 0,304    |
| Exp4 | 30      | 300     | 128        |   | 4        | 0,25    | VERDADERO     | 20     | VERDADERO | FastText  | 0,5        | 0,66           | 0,54   | 0,276    |
| Exp5 | 64      | 300     | 256        | 3 | 3        | 0,25    | VERDADERO     | 20     | VERDADERO | FastText  | 0,5        | 0,69           | 0,56   | 0,304    |
| Exp6 | 64      | 300     | 128        |   | 2        | 0,25    | VERDADERO     | 10     | VERDADERO | FastText  | 0,5        | 0,71           | 0,56   | 0,29     |
| Exp7 | 64      | 300     | 256        |   | 3        | 0,1     | VERDADERO     | 10     | FALSO     | FastText  | 0,57       | 0,7            | 0,61   | 0,219    |
| Exp8 | 64      | 300     | 256        |   | 3        | 0,3     | VERDADERO     | 20     | VERDADERO | FastText  | 0,51       | 0,69           | 0,56   | 0,295    |

Mejores Resultados


|      | batches | emb_dim | hidden_dim |  | n_layers | dropout | bidirectional | epochs | freeze | Embedding | Val Recall | Val Precission | Val F1 | Val Loss |
|------|---------|---------|------------|--|----------|---------|---------------|--------|--------|-----------|------------|----------------|--------|----------|
| Exp9 | 64      | 300     | 256        |  | 3        | 0,1     | VERDADERO     | 10     | FALSO  | FastText  | 0,57       | 0,7            | 0,61   | 0,219    |

Resumen Mejores Resultados

| Model         | Exp# | batches | emb_dim | hidden_dim | n_layers | dropout | bidirectional | epochs | freeze    | Embedding | Val Recall | Val Precission | Val F1 | Val Loss |
|---------------|------|---------|---------|------------|----------|---------|---------------|--------|-----------|-----------|------------|----------------|--------|----------|
| LSTM emb pree | Exp5 | 30      | 300     | 128        | 7        | 0,25    | VERDADERO     | 20     | VERDADERO | Glove     | 0,56       | 0,69           | 0,59   | 0,245    |
| GRU emb pree  | Exp5 | 64      | 300     | 256        | 3        | 0,1     | VERDADERO     | 10     | FALSO     | FastText  | 0,57       | 0,7            | 0,61   | 0,219    |
| LSTM          | Exp5 | 64      | 100     | 256        | 5        | 0,25    | VERDADERO     | 10     | -         | -         | 0,62       | 0,69           | 0,64   | 0,265    |
| GRU           | Exp7 | 64      | 300     | 256        | 3        | 0,25    | VERDADERO     | 10     | -         | -         | 0,63       | 0,74           | 0,66   | 0,182    |

## Conclusiones


Como se puede observar en la tabla de resumen de la sección de mejores resultados, los mejores resultados obtenidos fueron para el modelo RNN tipo GRU sin embeddings preentrenados, seguido de cerca por el modelo RNN tipo LSTM sin embeddings preentrenados. Estos modelos se alejan en primera instancia de los resultados obtenidos con sus símiles con embeddings preenrtenados. 

Esta diferencia entre arquitecturas es un resultado inesperado para el equipo de desarrollo, en tanto que el uso de embeddings preentrenados tenía la intención de mejorar los resultados obtenidos sin ellos. Las posibles explicaciones son las eventuales diferencias entre el dataset sobre el que se entrenó en este informe versus la fuente sobre la que se entrenaron los embeddings preentrenados utilizados. Y también que la forma en que se inicializaron pudo no ser la óptima para aprovechar sus características.

Con respecto a las métricas de evaluación, notamos que los resultados obtenidos tienden a sobresalir en Precision pero a bajar en Recall. La explicación tiene que ver, primero, con el trade-off entre estas dos métricas de evaluación; y segundo en que los resultados mostraron mayor capacidad de acertar una etiqueta correctamente.
Por esta razón, el F1-Score tendía a mantenerse al avanzar los experimentos, debido a que mejorar aún más el Precision perjudicaba el Recall y viceversa.

En lo que concierne a los hiperparámetros que se variaron para esta tarea, la primera conclusión es que la bidireccionalidad ayudó a mejorar los resultados (fue una de las primeras variables que fueron claras en los experimentos preliminares durante la construcción de las arquitecturas), debido a que se hace cargo del contexto en que aparecen las palabras. Luego, que para las arquitecturas sin embeddings preentrenados con 10 épocas bastaba para obtener buenos resultados sin overfitting, mientras que con embeddings preentrenados a veces se nececsitaba llegar a 20 para mejorar los resultados. Un dropout de 0,25 ayudó a mantener estables los resultados, ya que cuando se varió no se obtuvieron mejores resultados controlando el overfitting, que es la misión del dropout. En general aumentar la cantidad de layers de la RNN fue productivo, pero hasta cierto punto, por lo general mejoraba los resultados en las primeras interaciones, como también lo hacía aumentar las dimensiones ocultas. Finalmente, para embeddings no preentrenados, aumentar el tamaño a más de 300 no mostró mejorar los resultados, pero sí mejoró con respecto a 100.

Antes de proponer trabajo futuro cabe mencionar que el optimizer no se varió, pues la investigación previa mostró que Adam tendía a ser la elección para este tipo de problema y muchos otros en NLP. Por eso se decidió usar Adam a lo largo de todos los experimentos, pero sin lugar a dudas sería provechoso porbar otros optimizadores, y por eso se deja propuesto para futuros trabajos.
También se propone variar la función de Loss, ya que en este trabajo solamente se utilizó una CrossEntropyLoss dado que experimentos preliminares durante la creación de arquitectura mostraron resultados menos que aceptables con otras funciones que se probaron. Se propone investigar las funciones de Loss más usadas para NER para poder ser usadas como un parámetro a variar en este trabajo. También se propone probar diferentes embeddings preentrenados en españos, haciendo uso de BERT y Flair, para probar mejorar las performance de los usados en este informe, e investigar diferentes maneras de incorporarlos al modelo.

Finalmente, concluimos que se cumplieron los objetivos, pues se mejoraron los resultados inciales del baseline entregado por el equipo docente en las tres métricas a evaluar, se utilizaron diferentes herramientas para poder optmizar los resultados y se propusieron mejoras y experimentos para trabajos futuros en esta misma tarea.



## Anexos

1. Tabla completa de resultados obtenidos por cada experimento con tablas de resumen general y para cada experimento: 

https://docs.google.com/spreadsheets/d/1rF5ApuZYQ_dXKC8vitxjzhDIylq_DvVRpCIgfbMA9WM/edit?usp=sharing